In [190]:
# configuración para notebook con instalación LOCAL
FUENTES_DIR  = '../Fuentes'     # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR    = '../Datos/'      # carpeta donde se encuentran los datasets
MODELOS_DIR  = '../Modelos/'    # carpeta donde se encuentran los modelos
ARCHIVOs_DIR = '../Archivos/'   # carpeta recuperar o copiar archivos

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

In [191]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection
from matplotlib import pylab as plt

from grafica import *
from ClassNeuronaGral import NeuronaGradiente

In [192]:
import chardet


nombre_archivo = DATOS_DIR + 'SEMILLAS.csv' 

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv(nombre_archivo, encoding=result['encoding'])

# visualiza el dataframe

df

,Area,Perimetro,Compacidad,LongNucleo,AnchoNucleo,Asimetria,LongSurco,Clase
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,Tipo1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,Tipo1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,Tipo1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,Tipo1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,Tipo1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,Tipo3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,Tipo3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,Tipo3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,Tipo3


In [193]:
X_raw = np.array(df.iloc[:,0:-1])
T_raw = np.array(df.iloc[:,-1])

clases = np.unique(T_raw)
print(clases)

scaler = preprocessing.StandardScaler()
#X = scaler.fit_transform(X_raw)

['Tipo1' 'Tipo2' 'Tipo3']


In [194]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, T_raw, test_size=0.40, random_state=42)

In [195]:
# # neurona no lineal sigmoid
# ERROR = 1e-03
# ALFA = 0.2
# FUN = 'sigmoid'
# MAX_ITE = 200

# # hace entrenamiento por cada clase
# for cls_name in clases:
#     # Establece resultado esperado según la clase. 1=clase esperada, 0=otra clase

#     T = 1 * (y_train == cls_name)

#     # Entrena perceptrón
#     modelo = NeuronaGradiente(alpha=ALFA, n_iter=MAX_ITE, FUN=FUN, cotaE=ERROR)
#     modelo.fit(X_train, T)
#     (W, b, iteraciones) = (modelo.w_, modelo.b_, len(modelo.errors_))

#     Y = modelo.predict(X_train)
#     y_test = 1 * (y_test == cls_name)
#     efectividad = 100.0*np.sum(Y==y_train)/len(Y)

#     print("Clase %s con %6.2f%% de efectividad con %d iteraciones" % (cls_name, efectividad, iteraciones ))

# -----------------------------------------------------

In [196]:
# neurona no lineal sigmoid
ERROR = 1e-03
ALFA = 0.1
FUN = 'sigmoid'
MAX_ITE = 300
CORRIDAS = 30
TEST_SIZE = .4
print('\nNeurona con %s' %FUN)

cls_name = 'Tipo2'
prom_ite = 0
prom_acc_train = 0
prom_acc_test = 0
veces_train = 0
veces_test = 0

if (FUN == 'tanh'):
    T = 2*(T_raw==cls_name)-1
else:
    T = (T_raw==cls_name)*1

for nro_ite in range(CORRIDAS):
    # Establece resultado esperado según la clase. 1=clase esperada, 0=otra clase

    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_raw, T, test_size=TEST_SIZE, random_state=42)

    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)

    # Entrena perceptrón
    modelo = NeuronaGradiente(alpha=ALFA, n_iter=MAX_ITE, FUN=FUN, cotaE=ERROR)
    modelo.fit(X_train, Y_train)

    (W, b, iteraciones) = (modelo.w_, modelo.b_, len(modelo.errors_))

    prom_ite = prom_ite + iteraciones

    # efectividad entrenamiento
    Y_pred = modelo.predict(X_train)
    efectividad = 100.0*np.sum(Y_train==Y_pred)/len(Y_pred)
    prom_acc_train = prom_acc_train + efectividad
    if (100-efectividad<0.000001):
        veces_train = veces_train + 1

    print("Clase %s (E) con %6.2f%% de efectividad con %d iteraciones" % (cls_name, efectividad, iteraciones ))

    # efectividad prueba
    Y_pred = modelo.predict(X_test)
    efectividad = 100.0*np.sum(Y_test==Y_pred)/len(Y_pred)
    prom_acc_test = prom_acc_test + efectividad
    if (100-efectividad<0.000001):
        veces_test = veces_test + 1


print('promedio iteraciones  : %6.2f' % (prom_ite/CORRIDAS))
print('promedio acc. train   : %6.2f' % (prom_acc_train/CORRIDAS))
print('promedio acc. test    : %6.2f' % (prom_acc_test/CORRIDAS))
print('ejec. train con 100%%  : %3d' % (veces_train))
print('ejec.  test con 100%%  : %3d' % (veces_test))


Neurona con sigmoid


Clase Tipo2 (E) con  99.21% de efectividad con 289 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 285 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 285 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 276 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 288 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 280 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 279 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 280 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 280 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 269 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 283 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 280 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 278 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 280 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 273 iteraciones
Clase Tipo2 (E) con  99.21% de efectividad con 271 iter

In [197]:
# neurona no lineal tanh
ERROR = 1e-03
ALFA = 0.1
FUN = 'tanh'
MAX_ITE = 300
CORRIDAS = 30
TEST_SIZE = .4
print('\nNeurona con %s' %FUN)

cls_name = 'Tipo2'
prom_ite = 0
prom_acc_train = 0
prom_acc_test = 0
veces_train = 0
veces_test = 0

if (FUN == 'tanh'):
    T = 2*(T_raw==cls_name)-1
else:
    T = (T_raw==cls_name)*1

for nro_ite in range(CORRIDAS):
    # Establece resultado esperado según la clase. 1=clase esperada, 0=otra clase

    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_raw, T, test_size=TEST_SIZE, random_state=42)

    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)

    # Entrena perceptrón
    modelo = NeuronaGradiente(alpha=ALFA, n_iter=MAX_ITE, FUN=FUN, cotaE=ERROR)
    modelo.fit(X_train, Y_train)

    (W, b, iteraciones) = (modelo.w_, modelo.b_, len(modelo.errors_))

    prom_ite = prom_ite + iteraciones

    # efectividad entrenamiento
    Y_pred = modelo.predict(X_train)
    efectividad = 100.0*np.sum(Y_train==Y_pred)/len(Y_pred)
    prom_acc_train = prom_acc_train + efectividad
    if (100-efectividad<0.000001):
        veces_train = veces_train + 1

    print("Clase %s (E) con %6.2f%% de efectividad con %d iteraciones" % (cls_name, efectividad, iteraciones ))

    # efectividad prueba
    Y_pred = modelo.predict(X_test)
    efectividad = 100.0*np.sum(Y_test==Y_pred)/len(Y_pred)
    prom_acc_test = prom_acc_test + efectividad
    if (100-efectividad<0.000001):
        veces_test = veces_test + 1


print('promedio iteraciones  : %6.2f' % (prom_ite/CORRIDAS))
print('promedio acc. train   : %6.2f' % (prom_acc_train/CORRIDAS))
print('promedio acc. test    : %6.2f' % (prom_acc_test/CORRIDAS))
print('ejec. train con 100%%  : %3d' % (veces_train))
print('ejec.  test con 100%%  : %3d' % (veces_test))


Neurona con tanh
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efectividad con 300 iteraciones
Clase Tipo2 (E) con 100.00% de efecti